In [119]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
#import seaborn as sns
#from scipy.optimize import curve_fit
#from scipy import stats
#from collections import Counter
#from matplotlib import patheffects
#import matplotlib.patches as patches

In [120]:
df_niveles_carlos = pd.read_excel(r"C:\Users\gpavez\Desktop\Compensaciones\git\compensaciones\niveles_cramer\Estructura 2025_ Cramer.xlsx", sheet_name="Reporte de Estructura")
df_niveles_deborah = pd.read_excel(r"C:\Users\gpavez\Desktop\niveles deborah.xlsx")

c:\Users\gpavez\Desktop\Compensaciones\git\compensaciones\entorno_compensaciones\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [121]:
df_niveles_carlos_limpio = df_niveles_carlos.drop(['Empleado - Estado', 'Empleado - Nombre Completo',
       'Trabajo - Centro de Costo', 'Trabajo - ID Área',
       'Centro de Costo Empresa','Unidad',
       'Empleado - Nacionalidad', "SUELDO BASE",'Empleado - Fecha de Nacimiento','Empleado - Edad','Empleado - Estado Civil', 'Job Family', 'Trabajo - Dirección Área', 'Trabajo - Familia de Cargo',
       'Trabajo - Nombre Supervisor', 'Trabajo - Cargo Supervisor',
       'Trabajo - Número de Documento Supervisor', 'Trabajo - Nombre Área', '100% BANDA', 'PRB', 'DETALLE', "Empresa - Nombre Empresa"], axis=1)
df_niveles_carlos_limpio = df_niveles_carlos_limpio.rename(columns={'Empleado - Número de Documento':'Rut', 'GRADO':'grado_carlos'})
# Limpiar formato del RUT más completo
def limpiar_rut(rut):
    if pd.isna(rut):
        return rut
    return str(rut).replace('.', '').replace(' ', '').strip()

df_niveles_carlos_limpio['Rut'] = df_niveles_carlos_limpio['Rut'].apply(limpiar_rut)


df_niveles_carlos_limpio.head()

,Rut,Trabajo - Cargo,grado_carlos
0,16083286-k,Operario,11
1,9491159-1,Operario,11
2,18957219-0,Abogado,16
3,15777356-9,Ayudante De Bodega,10
4,15797997-3,Ingeniero De Excelencia Operacional,16


In [122]:
df_niveles_deborah_limpio = df_niveles_deborah.drop(['Empresa - Nombre Empresa', 'Fecha de Nac', 'edad',
       'Fecha Ingreso', 'Años en la empresa', 'Cenco','Gratif', 'Mov','lider', 'criticidad',
       'aumento en enero ', 'Nuevo SB', '%', 'Obs'], axis=1)
df_niveles_deborah_limpio['Rut'] = df_niveles_deborah_limpio['Rut'].apply(limpiar_rut)
df_niveles_deborah_limpio.head()

,Rut,Nombre,Cargo,Sueldo Base,Nivel 24,Nivel 25
0,20051168-9,Aránguiz Palacios Matías Nicolás,Gestor de Servicios y Proyectos,2140000,NaN,NaN
1,25861292-2,Noriega Garcia Lisbeth Carami,Supervisor De Planta,1386000,NaN,NaN
2,26542918-1,Alvarado Magne Lisa,Asistente De Laboratorio,610000,4,NaN
3,21889457-7,Ramos Velásquez Fernando Ignacio,Ayudante De Bodega,550000,NaN,NaN
4,19683409-5,Mercado González Francisca Javiera,Asistente De Laboratorio,610000,NaN,NaN


In [123]:
# Crear columna de nivel unificado
def unificar_niveles(row):
    nivel_24 = row['Nivel 24']
    nivel_25 = row['Nivel 25']
    
    # Si ambos son NaN, mantener NaN
    if pd.isna(nivel_24) and pd.isna(nivel_25):
        return np.nan
    
    # Si solo nivel 24 tiene valor, usar nivel 24
    if pd.notna(nivel_24) and pd.isna(nivel_25):
        return nivel_24
    
    # Si solo nivel 25 tiene valor, usar nivel 25
    if pd.isna(nivel_24) and pd.notna(nivel_25):
        return nivel_25
    
    # Si ambos tienen valor y son diferentes, priorizar nivel 25
    if pd.notna(nivel_24) and pd.notna(nivel_25):
        return nivel_25
    
    return np.nan

# Aplicar la función para crear la columna unificada
df_niveles_deborah_limpio['grado_deborah'] = df_niveles_deborah_limpio.apply(unificar_niveles, axis=1)

# Mostrar algunos ejemplos
#df_niveles_deborah_limpio[["Rut","Cargo", "Sueldo Base", 'grado_deborah']].head()
df_niveles_deborah_limpio = df_niveles_deborah_limpio.drop(columns=['Nivel 24', 'Nivel 25'])

df_niveles_deborah_limpio.head()

,Rut,Nombre,Cargo,Sueldo Base,grado_deborah
0,20051168-9,Aránguiz Palacios Matías Nicolás,Gestor de Servicios y Proyectos,2140000,NaN
1,25861292-2,Noriega Garcia Lisbeth Carami,Supervisor De Planta,1386000,NaN
2,26542918-1,Alvarado Magne Lisa,Asistente De Laboratorio,610000,4
3,21889457-7,Ramos Velásquez Fernando Ignacio,Ayudante De Bodega,550000,NaN
4,19683409-5,Mercado González Francisca Javiera,Asistente De Laboratorio,610000,NaN


In [124]:
# Opción 3: Outer join (todos los registros de ambos DataFrames)
df_unificado = pd.merge(df_niveles_carlos_limpio, df_niveles_deborah_limpio, 
                        on='Rut', how='outer', suffixes=('_carlos', '_deborah'))

# Crear columna de cargo unificado
def unificar_cargo(row):
    cargo_carlos = row['Trabajo - Cargo']
    cargo_deborah = row['Cargo']
    
    # Si ambos son NaN, mantener NaN
    if pd.isna(cargo_carlos) and pd.isna(cargo_deborah):
        return np.nan
    
    # Si solo uno tiene valor, usar ese
    if pd.notna(cargo_carlos) and pd.isna(cargo_deborah):
        return cargo_carlos
    
    if pd.isna(cargo_carlos) and pd.notna(cargo_deborah):
        return cargo_deborah
    
    # Si ambos tienen valor y son iguales, usar cualquiera
    if str(cargo_carlos).strip().lower() == str(cargo_deborah).strip().lower():
        return cargo_carlos
    
    # Si son diferentes, concatenar ambos para revisión manual
    return f"{cargo_carlos} | {cargo_deborah}"

# Aplicar la función
df_unificado['cargo_unificado'] = df_unificado.apply(unificar_cargo, axis=1)

# Ver discrepancias (donde los cargos son diferentes)
discrepancias_cargo = df_unificado[
    (df_unificado['Trabajo - Cargo'].notna()) & 
    (df_unificado['Cargo'].notna()) & 
    (df_unificado['Trabajo - Cargo'].str.strip().str.lower() != 
     df_unificado['Cargo'].str.strip().str.lower())
][['Rut', 'Trabajo - Cargo', 'Cargo', 'cargo_unificado']]

# print(f"Discrepancias encontradas en cargos: {len(discrepancias_cargo)}")
# if len(discrepancias_cargo) > 0:
#     print("\nPrimeras 10 discrepancias:")
#     print(discrepancias_cargo.head(10))

# Estadísticas de completitud
print(f"\nEstadísticas de completitud:")
print(f"Trabajo - Cargo (Carlos): {df_unificado['Trabajo - Cargo'].notna().sum()} de {len(df_unificado)}")
print(f"Cargo (Deborah): {df_unificado['Cargo'].notna().sum()} de {len(df_unificado)}")
print(f"Cargo Unificado: {df_unificado['cargo_unificado'].notna().sum()} de {len(df_unificado)}")

# Mostrar las primeras filas con la nueva columna
df_unificado.head()


Estadísticas de completitud:
Trabajo - Cargo (Carlos): 534 de 574
Cargo (Deborah): 543 de 574
Cargo Unificado: 574 de 574


,Rut,Trabajo - Cargo,grado_carlos,Nombre,Cargo,Sueldo Base,grado_deborah,cargo_unificado
0,10033163-2,Jefe Unidad Investigación y Sensorial,17.0,Piderit Gonzalez Veronica Daniela,Jefe Unidad Investigación y Sensorial,4000000.0,10,Jefe Unidad Investigación y Sensorial
1,10036334-8,NaN,NaN,Schwartz Greve Andrea,Gerente Comercial Sabores Chile,10400000.0,NaN,Gerente Comercial Sabores Chile
2,10172861-7,NaN,NaN,Ulloa Bozo Michel Patricio,Gerente Comercial Fragancias,11000000.0,NaN,Gerente Comercial Fragancias
3,10179598-5,Perfumista,16.0,Muñoz Infante Patricia Jimena,Perfumista,4900000.0,13,Perfumista
4,10279910-0,Key Account Manager,15.0,Catalan Pino Jacquelinne Victoria,Key Account Manager,4650000.0,12,Key Account Manager


In [125]:
columnas_ordenadas = ['Rut', "Nombre", 'Cargo', "Trabajo - Cargo", "Sueldo Base", 'grado_deborah', 'grado_carlos']
df_unificado_ordenado = df_unificado[columnas_ordenadas]
df_unificado_ordenado.head()


,Rut,Nombre,Cargo,Trabajo - Cargo,Sueldo Base,grado_deborah,grado_carlos
0,10033163-2,Piderit Gonzalez Veronica Daniela,Jefe Unidad Investigación y Sensorial,Jefe Unidad Investigación y Sensorial,4000000.0,10,17.0
1,10036334-8,Schwartz Greve Andrea,Gerente Comercial Sabores Chile,NaN,10400000.0,NaN,NaN
2,10172861-7,Ulloa Bozo Michel Patricio,Gerente Comercial Fragancias,NaN,11000000.0,NaN,NaN
3,10179598-5,Muñoz Infante Patricia Jimena,Perfumista,Perfumista,4900000.0,13,16.0
4,10279910-0,Catalan Pino Jacquelinne Victoria,Key Account Manager,Key Account Manager,4650000.0,12,15.0


In [126]:
# Función para convertir números romanos a enteros
def romano_a_entero(romano):
    valores_romanos = {
        'I': 1, 'II': 2, 'III': 3, 'IV': 4, 'V': 5,
        'VI': 6, 'VII': 7, 'VIII': 8, 'IX': 9, 'X': 10,
        'XI': 11, 'XII': 12, 'XIII': 13, 'XIV': 14, 'XV': 15,
        'XVI': 16, 'XVII': 17, 'XVIII': 18, 'XIX': 19, 'XX': 20
    }
    return valores_romanos.get(str(romano).upper(), None)

# Función para convertir cualquier valor a entero (números o romanos)
def convertir_a_entero(valor):
    if pd.isna(valor):
        return None
    
    # Intentar convertir directamente a entero
    try:
        return int(float(valor))
    except (ValueError, TypeError):
        # Si falla, intentar como número romano
        romano_convertido = romano_a_entero(valor)
        if romano_convertido is not None:
            return romano_convertido
        else:
            # Si no es ni número ni romano válido, retornar None
            return None

# Crear análisis de niveles por cargo separado para Deborah y Carlos
def obtener_niveles_por_cargo_separado(df):
    resultado = []
    
    for cargo in df['Cargo'].dropna().unique():
        # Filtrar por cargo
        df_cargo = df[df['Cargo'] == cargo]
        
        # Obtener niveles únicos de Deborah
        niveles_deborah = df_cargo['grado_deborah'].dropna().unique()
        niveles_deborah_convertidos = [convertir_a_entero(x) for x in niveles_deborah]
        # Usar set() para eliminar duplicados antes de ordenar
        niveles_deborah_unicos = sorted(list(set([x for x in niveles_deborah_convertidos if x is not None])))
        niveles_deborah_str = ','.join(map(str, niveles_deborah_unicos)) if niveles_deborah_unicos else 'Sin niveles'
        
        # Obtener niveles únicos de Carlos
        niveles_carlos = df_cargo['grado_carlos'].dropna().unique()
        niveles_carlos_convertidos = [convertir_a_entero(x) for x in niveles_carlos]
        # Usar set() para eliminar duplicados antes de ordenar
        niveles_carlos_unicos = sorted(list(set([x for x in niveles_carlos_convertidos if x is not None])))
        niveles_carlos_str = ','.join(map(str, niveles_carlos_unicos)) if niveles_carlos_unicos else 'Sin niveles'
        
        resultado.append({
            'cargo': cargo,
            'niveles_deborah': niveles_deborah_str,
            'niveles_carlos': niveles_carlos_str
        })
    
    return pd.DataFrame(resultado)

# Crear el nuevo DataFrame con columnas separadas
df_niveles_separados = obtener_niveles_por_cargo_separado(df_unificado_ordenado)
df_niveles_separados.head(10)

,cargo,niveles_deborah,niveles_carlos
0,Jefe Unidad Investigación y Sensorial,10,17
1,Gerente Comercial Sabores Chile,Sin niveles,Sin niveles
2,Gerente Comercial Fragancias,Sin niveles,Sin niveles
3,Perfumista,13,16
4,Key Account Manager,"7,8,10,12",15
5,Operario,"1,2,3,4,5",11
6,Subgerente Control y Cumplimiento Financiero,13,18
7,Asistente De Servicios Generales,"1,2,5",10
8,Jefe De Credito Y Cobranzas,10,17
9,Asistente De Bodega,3,10


In [127]:
# Exportar ambos DataFrames a Excel con múltiples hojas
with pd.ExcelWriter(r"C:\Users\gpavez\Desktop\Compensaciones\git\compensaciones\niveles_unificados.xlsx", 
                    engine='openpyxl') as writer:
    
    # Hoja 1: Datos unificados (datos originales)
    df_unificado_ordenado.to_excel(writer, 
                                   sheet_name='Niveles_Unificados', 
                                   index=False)
    
    # Hoja 2: Resumen de niveles por cargo
    df_niveles_separados.to_excel(writer, 
                                  sheet_name='Resumen_Niveles_Por_Cargo', 
                                  index=False)

print("Archivo Excel exportado exitosamente!")
print(f"Ubicación: C:\\Users\\gpavez\\Desktop\\Compensaciones\\git\\compensaciones\\niveles_unificados.xlsx")
print(f"Hoja 1 - Niveles_Unificados: {len(df_unificado_ordenado)} registros")
print(f"Hoja 2 - Resumen_Niveles_Por_Cargo: {len(df_niveles_separados)} registros")
print("Hojas creadas:")
print("  - Niveles_Unificados: Datos detallados por persona")
print("  - Resumen_Niveles_Por_Cargo: Niveles únicos agrupados por cargo")

Archivo Excel exportado exitosamente!
Ubicación: C:\Users\gpavez\Desktop\Compensaciones\git\compensaciones\niveles_unificados.xlsx
Hoja 1 - Niveles_Unificados: 574 registros
Hoja 2 - Resumen_Niveles_Por_Cargo: 157 registros
Hojas creadas:
  - Niveles_Unificados: Datos detallados por persona
  - Resumen_Niveles_Por_Cargo: Niveles únicos agrupados por cargo
